### Install `LangChain` library

Please run the following script if you do not have the library LangChain installed in your computer. For more details, please visit [LangChain doumentation](https://python.langchain.com/en/latest/getting_started/getting_started.html).

In [ ]:
!pip3 install langchain

### Instantiate an LLM

LangChain supports a variety of LLM providers which can be found in [this link](https://python.langchain.com/en/latest/modules/models/llms/integrations.html). The model `google/flan-t5-xl` from [Hugging Face](https://huggingface.co/) will be used as the main LLM throughout the notebooks (please prepare [Hugging Face's API token](https://huggingface.co/docs/hub/security-tokens#:~:text=To%20create%20an%20access%20token,you're%20ready%20to%20go!) to access the model). However, you can change the model to other LLMs, such as [OpenAI](https://python.langchain.com/en/latest/modules/models/llms/integrations/openai.html), to obtain better experience.

In [1]:
import os
from langchain import HuggingFaceHub

In [2]:
# replace with your token here.
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [3]:
# initialize LLM.
llm = HuggingFaceHub(
    repo_id = "google/flan-t5-xl",
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    model_kwargs = {
        "temperature": 0
    }
)

### Try it with your query

In [4]:
llm("Translate 'I love you' in French")

"Je t'aime"

In [5]:
question = "What is LangChain?"
llm(question)

'LangChain is a blockchain platform that allows users to create and manage their own private'

What? LangChain is now a blockchain platform (LOL). Please note that language models are simply probability models that use the input as a condition to select what comes next based on the highest conditional probability. This issue may be attributed to the limited datasets used for training the model. However, we can improve the model performance by leveraging in-context learning, where a `prompt` is attached to guide the model to behave more reasonably in the desired task.

### Empower the LLM with prompt

A possible solution is to provide the model with additional context or knowledge in the form of a prompt. This allows the model to incorporate the new information into its generation process which hopefully improves the quality of its answers.

For implementation, LangChain provides `PromptTemplate` which is a powerful tool to manage a prompt. Here is an example of the implementation.

In [6]:
from langchain import PromptTemplate, LLMChain

In the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html), it requires two essential ingredients to create a prompt. These are `template` and `input_variables`.
- `template` is a text used to provide additional information to the model, such as instructions and few-shot example. It may also contain placeholders for flexibility, allowing for the creation of prompts associated with specific questions or tasks.
- `input_variables` is a list of variables to replace with the placeholders used in the `template`.

In [7]:
template = """
Given the following extracted parts of a long document and a question, create a final answer. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
QUESTION: {question}
=========
{context}
=========
FINAL ANSWER: 
"""

In this examples, `template` contains two placeholders which are `question` and `context`, meaning that `input_variables` is `["question", "context"]`.

In [8]:
PROMPT = PromptTemplate(
    template = template,
    input_variables = ["question", "context"],
)

We can attach the prompt to LLM using `LLMChain` by specifying three main inputs, namely
- `llm`: a large language model we would like to deploy.
- `prompt`: a `PromptTemplate`.
- `output_key`: a key in the dictionary to represent the output text (the output of `LLMChain` is designed as a dictionary).

In [9]:
llm_chain = LLMChain(
    llm = llm,
    prompt = PROMPT,
    output_key = "answer"
)

### Generate an answer

In the `PromptTemplate`, the variable `context` can be used to provide additional context to the model. In this case, we may create context for the LangChain here since it is relevant to the question.

In [10]:
context = """
LangChain is a framework for developing applications powered by language models. 
We believe that the most powerful and differentiated applications will not only call out 
to a language model via an API, but will also:
- Be data-aware: connect a language model to other sources of data
- Be agentic: allow a language model to interact with its environment
The LangChain framework is designed with the above principles in mind.
"""

In [11]:
output = llm_chain(
    {
        "question": question,
        "context": context
    }
)

print("Q:", question)
print("A:", output["answer"])

Q: What is LangChain?
A: a framework for developing applications powered by language models


Here we are! The model is now able to generate the answer more accurately with the additional context provided.